<a href="https://colab.research.google.com/github/caendeith/mlBTC/blob/main/mlBTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Загрузка данных
df = pd.read_csv('/content/drive/MyDrive/btcusd_1-min_data.csv')
df['Date'] = pd.to_datetime(df['Timestamp'], unit='s')
df = df[['Date', 'Close']].sort_values(by='Date')

# Нормализация
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))

print("Данные нормализованы")

Данные нормализованы


In [14]:
def create_sequences(data, look_back=60):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, look_back=60)
X = X.reshape((X.shape[0], X.shape[1], 1))  # для LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Создаем модель
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

# Компиляция
model.compile(optimizer='adam', loss='mse')

# Обучение
model.fit(X, y, epochs=10, batch_size=32, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/10
220850/220850 ━━━━━━━━━━━━━━━━━━━━ 12084s 55ms/step - loss: 1.8414e-05
Epoch 2/10
220850/220850 ━━━━━━━━━━━━━━━━━━━━ 12566s 57ms/step - loss: 3.4565e-07
Epoch 3/10
220850/220850 ━━━━━━━━━━━━━━━━━━━━ 13611s 62ms/step - loss: 2.4710e-07
Epoch 4/10
  2036/220850 ━━━━━━━━━━━━━━━━━━━━ 4:04:10 67ms/step - loss: 1.7521e-07

In [ ]:
# Последние N дней для прогноза
last_sequence = scaled_data[-60:]  # 60 дней назад
inputs = last_sequence.copy()

forecast = []

for _ in range(7):
    prediction = model.predict(inputs[np.newaxis, :, :], verbose=0)
    forecast.append(prediction[0, 0])
    inputs = np.append(inputs[1:], prediction, axis=0)

# Обратное преобразование
forecast_prices = scaler.inverse_transform(np.array(forecast).reshape(-1, 1)).flatten()

# Выводим прогноз
dates = pd.date_range(df['Date'].max(), periods=8, freq='D')[1:]
print("\nПрогноз цены Bitcoin на неделю вперед:")
for d, p in zip(dates, forecast_prices):
    print(f"{d.strftime('%Y-%m-%d')}: ${p:.2f}")